# Funciones de distribución y orden de los datasets

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import join
import re

In [ ]:
def create_chord_annotation_database(path, frame_length, maj_min=True):
    print("Reading chord annotation data from %s" %(path))
    chord_folders =  sorted(listdir(path))
    chord_folders = [folder for folder in chord_folders if not folder.endswith('.txt')]
    df = pd.DataFrame()
    
    for folder in chord_folders:
        chord_files = [file for file in sorted(listdir(join(path, folder, 'chord_annotations')))
                       if file.endswith(".lab")]
        # cont = 0
        for file in chord_files:
            tmp = pd.read_csv(join(path, folder, 'chord_annotations', file), sep='\s+', names=["tstart", "tend",
                                                                                               "chord"])
            # print(tmp.tstart)
            # cont += 1
            # if cont > 1:
            #     break
            album = folder
            track = file
            # print(album)
            # print(track)
            n_frames_per_chord = (tmp.tend - tmp.tstart) / frame_length
            res = n_frames_per_chord - round(n_frames_per_chord)
            df_zero = pd.Series([0])
            df_zero.rename(index={0: int(-1)}, inplace=True)
            res_shifted = pd.Series(pd.concat([df_zero, res[:-1]], axis=0))
            time = np.load(join('Database', album, 'timevector', track[:-4] + '_(prueba).npy'))
            n_frames_from_timevector = pd.Series()
            for ncont, (tstart, tend) in enumerate(zip(tmp.tstart, tmp.tend)):
                # print(tstart, tend)
                frames_within_timerange = [n for n, t in enumerate(time) if t >= tstart and t < tend]
                time_within_timerange = [t for n, t in enumerate(time) if t >= tstart and t < tend]
                num_frames = pd.Series(len(time_within_timerange))
                num_frames = num_frames.rename(lambda x: x + ncont)
                n_frames_from_timevector = pd.concat([n_frames_from_timevector, num_frames])
            # print(len(n_frames_from_timevector))
            # n_frames_from_timevector = n_frames_from_timevector.rename(lambda x: x + idx for idx in 
            #                                                           range(len(n_frames_from_timevector)))
            # print(n_frames_from_timevector)
            # n_frames_from_timevector = len(time_within_timerange)
            # framess = pd.concat([n_frames_per_chord, round(n_frames_per_chord), 
            #                      res, res_shifted],
            #                      axis=1)
            # res_shifted.index.names = np.arange(len(res))
            # res_shifted.rename(index=np.arange(len(res)), inplace=True)
            # res_shifted.rename(np.arange(len(res)))
            # res_shifted.rename(index={0:'zero',1:'one'}, inplace=True)
            res_shifted = res_shifted.rename(lambda x: x + 1)
            changed_frames = n_frames_per_chord + 0 * (0.125 / 2) * res_shifted 
            # print(res_shifted)
            # print(changed_frames)
            # res_shifted
            # print(framess)
            # framess
            # print(framesss, np.round(framesss))
            tmp = pd.concat([pd.DataFrame([folder] * len(tmp), columns=['disk']),
                                 pd.DataFrame([file] * len(tmp), columns=['track']),
                                 tmp,
                                 # pd.DataFrame((tmp.tend - tmp.tstart) / frame_length, dtype='int', columns=['nframes'])],
                                 # pd.DataFrame(round(changed_frames), dtype='int', columns=['nframes'])],
                                 pd.DataFrame(n_frames_from_timevector, dtype='int', columns=['nframes'])], 
                            axis=1)
            # print(pd.DataFrame(pd.concat([n_frames_per_chord, round(n_frames_per_chord),changed_frames, tmp['nframes']], axis=1)))
            # res = int(round((tmp.tend - tmp.tstart) / frame_length)) - (tmp.tend - tmp.tstart) / frame_length
            # tmp['nframes'] = tmp['nframes'].apply(lambda x: x + 1 if tmp.tstart == 0.0 and (tmp.tend - tmp.tstart) > 0.0)
            # Queda pendiente quitarle un frame al primer frame, cuando tstart = 0.0
            # print(tmp['nframes'])
            '''
            if tmp.tstart == 0.0:
                tmp = pd.concat([pd.DataFrame([folder] * len(tmp), columns=['disk']),
                                 pd.DataFrame([file] * len(tmp), columns=['track']),
                                 tmp,
                                 pd.DataFrame(2 * (tmp.tend - tmp.tstart) / frame_length, dtype='int', 
                                              columns=['nframes'])],
                                axis=1)
            else:
                tmp = pd.concat([pd.DataFrame([folder] * len(tmp), columns=['disk']),
                                 pd.DataFrame([file] * len(tmp), columns=['track']),
                                 tmp,
                                 pd.DataFrame((tmp.tend - tmp.tstart) / frame_length, dtype='int', columns=['nframes'])],
                                axis=1)
            '''
            df = df.append(tmp, ignore_index=True)
            # print(df)
    if maj_min:
        df.drop(df.index[df['chord'].str.contains('aug')], inplace=True)  # Eliminar aug del dataframe
        df.drop(df.index[df['chord'].str.contains('sus')], inplace=True)  # Eliminar sus del dataframe
        df.drop(df.index[df['chord'].str.contains('dim')], inplace=True)  # Eliminar dim del dataframe
    # Conversión de bemoles a sostenidos
    df['chord'] = df['chord'].apply(lambda x: 'C#' + x[2:] if 'Db' in x else x)
    df['chord'] = df['chord'].apply(lambda x: 'D#' + x[2:] if 'Eb' in x else x)
    df['chord'] = df['chord'].apply(lambda x: 'E' + x[2:] if 'Fb' in x else x)
    df['chord'] = df['chord'].apply(lambda x: 'F#' + x[2:] if 'Gb' in x else x)
    df['chord'] = df['chord'].apply(lambda x: 'G#' + x[2:] if 'Ab' in x else x)
    df['chord'] = df['chord'].apply(lambda x: 'A#' + x[2:] if 'Bb' in x else x)
    df['chord'] = df['chord'].apply(lambda x: 'B' + x[2:] if 'Cb' in x else x)
    if maj_min:
        # Eliminación de acordes que no son maj/min
        df.drop(df.index[df['chord'] == 'A#:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'A:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'A:(1,2,4)'], inplace=True)
        df.drop(df.index[df['chord'] == 'A:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'B:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'B:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'B:(5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'B:(6)'], inplace=True)
        df.drop(df.index[df['chord'] == 'B:(b3,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'C#:(1,b3)'], inplace=True)
        df.drop(df.index[df['chord'] == 'C#:(1,b3)/b3'], inplace=True)
        df.drop(df.index[df['chord'] == 'C:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'C:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'D#:(1,4)'], inplace=True)
        df.drop(df.index[df['chord'] == 'D:(*5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'D:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'D:(1,4)'], inplace=True)
        df.drop(df.index[df['chord'] == 'D:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'E:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'E:(1,2,5,b6)'], inplace=True)
        df.drop(df.index[df['chord'] == 'E:(1,4)'], inplace=True)
        df.drop(df.index[df['chord'] == 'E:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'E:(1,b7)'], inplace=True)
        df.drop(df.index[df['chord'] == 'E:(1,b7)/b7'], inplace=True)
        df.drop(df.index[df['chord'] == 'F#:(1,4,b5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'F#:(1,4,b7)'], inplace=True)
        df.drop(df.index[df['chord'] == 'F#:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'F:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'G#:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'G:(1)'], inplace=True)
        df.drop(df.index[df['chord'] == 'G:(1,5)'], inplace=True)
        df.drop(df.index[df['chord'] == 'G:(1,b3,4)'], inplace=True)
        df.drop(df.index[df['chord'] == 'G:(1,b3,4)/b3'], inplace=True)
        df.drop(df.index[df['chord'] == 'G:(6)'], inplace=True)
        df.drop(df.index[df['chord'] == 'G:(7)'], inplace=True)
        # df['chord'] = df['chord'].apply(lambda x: re.sub("b"+"\d+", "", x))
        # df['chord'] = df['chord'].apply(lambda x: re.sub("\d+" + "/" + "\d+", "", x))
        # df['chord'] = df['chord'].apply(lambda x: re.sub("/" + "\d+", "", x))
        # df['chord'] = df['chord'].apply(lambda x: x.replace('b', '').replace('h', '').replace('N:maj', 'N'))
        # df['chord'] = df['chord'].apply(lambda x: x.split(':')[0] + ":maj" if not x.split(':')[1] == 'min' else x)
        #
        df['chord'] = df['chord'].apply(lambda x: x.replace('minmaj', 'min'))
        df['chord'] = df['chord'].apply(lambda x: x.replace('F#min7', 'F#:min7'))  # Viene mal desde el ground-truth
        # df['chord'] = df['chord'].apply(lambda x: re.sub("/b" + "\d+", "", x))
        # df['chord'] = df['chord'].apply(lambda x: re.sub("\d+", "", x))
        df['chord'] = df['chord'].apply(lambda x: re.sub("/" + "\d+", "", x))  
        df['chord'] = df['chord'].apply(lambda x: re.sub("/b" + "\d+", "", x))
        df['chord'] = df['chord'].apply(lambda x: re.sub("/#" + "\d+", "", x))
        #
        df['chord'] = df['chord'].apply(lambda x: x + ':maj' if not ':' in x else x)
        # df['chord'] = df['chord'].apply(lambda x: x.replace('(b)', ''))
        # df['chord'] = df['chord'].apply(lambda x: x.replace('(', '').replace(')', '').replace(',', '.'))
        df['chord'] = df['chord'].apply(lambda x: x.split(':')[0] + ":min" if "min" in x.split(':')[1] else 
                                        x.split(':')[0] + ":maj")
        #df['chord'] = df['chord'].apply(lambda x: x[0:2] + 'm')
        df['chord'] = df['chord'].apply(lambda x: x.replace("N:maj", "N"))
    
    return df
    

In [ ]:
def stratified_set_split(df, rseed=None, train_ratio=0.7, val_ratio=0.2):
    print("Creating train, validation and test indices")
    test_ratio = 1. - train_ratio - val_ratio
    index_train, index_val, index_test = [], [], []
    chords_ocurrence = []
    unique_chords = sorted(df['chord'].unique())
    if not rseed is None:
        np.random.seed(rseed)
    for chord in unique_chords:
        frames = df.nframes[df.chord == chord]
        nframe_tot = frames.sum()
        chords_ocurrence.append((chord, nframe_tot))
        nframe_cum = frames.cumsum()
        RP = np.random.permutation(nframe_tot)
        for frame in RP[:int(train_ratio * nframe_tot)]:
            idx_cum = nframe_cum.searchsorted(frame)
            df_index = frames.index[idx_cum]
            last_cum = nframe_cum.iloc[idx_cum - 1] if idx_cum > 0 else 0
            index_train.append((df_index, frame - last_cum))
        for frame in RP[int(train_ratio * nframe_tot):int((train_ratio + val_ratio) * nframe_tot)]:
            idx_cum = nframe_cum.searchsorted(frame)
            df_index = nframe_cum.index[idx_cum]
            last_cum = nframe_cum.iloc[idx_cum - 1] if idx_cum > 0 else 0
            index_val.append((df_index, frame - last_cum))
        for frame in RP[int((train_ratio + val_ratio) * nframe_tot):]:
            idx_cum = nframe_cum.searchsorted(frame)
            df_index = nframe_cum.index[idx_cum]
            last_cum = nframe_cum.iloc[idx_cum - 1] if idx_cum > 0 else 0
            index_test.append((df_index, frame - last_cum))
    np.random.shuffle(index_train)
    np.random.shuffle(index_val)
    np.random.shuffle(index_test)
    
    return index_train, index_val, index_test, chords_ocurrence

In [ ]:
def stats_chords_ocurrence(ch_ocurrence):
    chords_order = ['C:maj', 'C:min', 'C#:maj', 'C#:min', 'D:maj', 'D:min', 'D#:maj', 'D#:min', 'E:maj', 'E:min', 
                    'F:maj', 'F:min', 'F#:maj', 'F#:min', 'G:maj', 'G:min', 'G#:maj', 'G#:min', 'A:maj', 'A:min', 
                    'A#:maj', 'A#:min', 'B:maj', 'B:min', 'N']
    order_dict = {t[0]:t[1] for t in list(ch_ocurrence)}
    chords_ocurrence = [order_dict[k] for k in chords_order]
    plt.rcParams['figure.figsize'] = [20, 5]
    ch_ocurr_percentage = [round(value / np.sum(chords_ocurrence) * 100, 2) for value in chords_ocurrence]
    with plt.rc_context({'axes.edgecolor':'white',  'axes.labelcolor': 'white', 'xtick.color':'white', 
                         'ytick.color':'white', 'figure.facecolor':'black'}):
        plt.bar(chords_order,  ch_ocurr_percentage)
    for i, v in enumerate(ch_ocurr_percentage):
        plt.text(chords_order[i], v + 0.15, "{:.2f}".format(v) + "%", 
                 color='black', ha='center', fontweight='bold')
    plt.xlabel('Acordes musicales')
    plt.ylabel('Porcentaje de ocurrencia')